In [ ]:
import altair as alt
import pandas as pd
from datetime import datetime
import openpyxl

In [ ]:
pip install openpyxl

In [ ]:
df=pd.read_excel('2014-2020.xlsx', engine='openpyxl',usecols=range(0,12))
df

In [ ]:
df['Year']=df['Час/Дата'].str[-4:]
df['Month']=df['Час/Дата'].str[-7:-5]
df['Day']=df['Час/Дата'].str.split('-',expand=True)[1].str[:2]
df['Time']=df['Час/Дата'].str.split('-',expand=True)[0]
df['date'] = df['Час/Дата'].apply(lambda x: datetime.strptime( x.split('-', 1)[1],'%d.%m.%Y'))
df['week day'] =pd.DatetimeIndex(df['date']).day_name()
df['day of the year'] =pd.DatetimeIndex(df['date']).strftime('%j')
df['season']=None
for index,row in df.iterrows():
    if row['Month'] in ['03', '04','05']:
        df['season'][index]='Spring'
    if row['Month'] in ['01', '02','12']:
        df['season'][index]='Winter'
    if row['Month'] in ['06', '07','08']:
        df['season'][index]='Summer'
    if row['Month'] in ['09', '10','11']:
        df['season'][index]='Fall'
df

In [ ]:
aes = df[['Time','Day', 'Year', 'date', 'AES']]
aes.columns = ['Time','Day', 'Year', 'date', 'energy']
aes.loc[:, 'category'] = 'AES'

tec = df[['Time','Day', 'Year', 'date', 'TEC']]
tec.columns = ['Time','Day', 'Year','date', 'energy']
tec.loc[:, 'category'] = 'TEC'

vde = df[['Time','Day', 'Year','date', 'VDE']]
vde.columns = ['Time','Day', 'Year', 'date', 'energy']
vde.loc[:, 'category'] = 'VDE'

tes = df[['Time','Day', 'Year', 'date', 'TES']]
tes.columns = ['Time','Day', 'Year', 'date', 'energy']
tes.loc[:, 'category'] = 'TES'

ges = df[['Time','Day', 'Year', 'date', 'GES']]
ges.columns = ['Time', 'Day', 'Year','date', 'energy']
ges.loc[:, 'category'] = 'GES'

gaes = df[['Time','Day', 'Year', 'date', 'GAES_GEN']]
gaes.columns = ['Time','Day', 'Year', 'date', 'energy']
gaes.loc[:, 'category'] = 'GAES'

df_long = pd.concat([aes, tec, vde, tes, ges, gaes])

In [ ]:
alt.data_transformers.disable_max_rows()

- Як змінювалась структура генерації електроенергії за роками?

Альтернативними варіантами у цьому випадку міг би бути bar chart. Проте мені здається що така структура була би більш громіздкою. Я вирішила використати area chart, щоб показати частку в цілому кожного джерела електроенергії а заодно зобразити зміну споживання енергії історично. 

In [ ]:
alt.Chart(df_long).transform_filter(alt.FieldRangePredicate(field='Year', range=[2014, 2020])).mark_area().encode(
    x = alt.X('Year:N', 
             scale = alt.Scale(
                  zero = False)),
    y = alt.Y('energy:Q', aggregate = 'sum'),
    color = alt.Color('category:N'),
    order = alt.Order('energy:Q', aggregate='sum', sort = 'ascending')
).properties(width = 950, height = 500, background = '#F9F9F9', padding = 25)

- Як залежить споживання електроенергії від дня року та години доби?

У цьому випадку я вирішила використати хітмап, щоб передати патерн використання електроенергії в залежності від двох заданих змінних. Також я застосувала спектральну кольорову схему, щоб краще передати різницю між низьким та високим використанням енергії. Недоліко у цьому графіку може бути власне спектральна схема, оскільки вона може на початку дезорієнтувати глядача.

In [ ]:
alt.Chart(df).mark_rect().encode(
    x = alt.X('day of the year:N'),
    y = alt.Y('Time:N', sort =  alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    color = alt.Color('CONSUMPTION:Q',scale = alt.Scale(scheme = 'spectral'))
).properties(background = '#F9F9F9', padding = 25)

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

У цьому випадку я вирішила використати нормалізований area chart,щоб показати яку частку від цілого займає кожне джерело енергій впродовж доби. Також тут можна було б використати bar chart. Недоліком у цьому випадку може бути те, що AES та GAES міняють позиції, що виглядає не зовсім естетично

In [ ]:
select_emissions = alt.selection_single(on = 'mouseover', fields=['category'], empty = 'all')
alt.Chart(df_long).transform_filter(alt.FieldRangePredicate(field='Time', range=[1, 24])).mark_area().encode(
    x = alt.X('Time:N', 
             scale = alt.Scale(
                  zero = False)),
    y = alt.Y('energy:Q', aggregate = 'sum', stack = 'normalize'),
    color = alt.Color('category:N'),
    opacity = alt.condition(
        select_emissions,
        alt.value(0.8),
        alt.value(0.3)
    ),
    order = alt.Order('energy:Q', aggregate='sum', sort = 'ascending')
).add_selection(select_emissions).properties(width = 950, height = 500, background = '#F9F9F9', padding = 25)

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

У цьому видаку я використала чотири хітмапи щоб зобразити різні пори року, а також передати залежність між місяцем та годиною доби. Альтернативою у цьому випадку міг би стати bar chart

In [ ]:
winter = alt.Chart(df[df['season'] == 'Winter']).mark_rect().encode(
    x = alt.X('Time:N', sort =  alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    y = alt.Y('Month:N'),
    color = alt.Color('CONSUMPTION:Q')
)

spring = alt.Chart(df[df['season'] == 'Spring']).mark_rect().encode(
    x = alt.X('Time:N', sort =  alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    y = alt.Y('Month:N'),
    color = alt.Color('CONSUMPTION:Q')
)

summer = alt.Chart(df[df['season'] == 'Summer']).mark_rect().encode(
    x = alt.X('Time:N', sort =  alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    y = alt.Y('Month:N'),
    color = alt.Color('CONSUMPTION:Q')
)

fall = alt.Chart(df[df['season'] == 'Fall']).mark_rect().encode(
    x = alt.X('Time:N', sort =  alt.Sort(field = 'Time', op = 'count', order = 'ascending')),
    y = alt.Y('Month:N'),
    color = alt.Color('CONSUMPTION:Q')
)

alt.vconcat(winter,spring,summer,fall).properties(background = '#F9F9F9', padding = 25)

- Як змінюється споживання електроенергії впродовж тижня?

У даному я випадку я випішила використати bar chart, оскільки він компактний та контрастно показує різницю між кількома значенням. Також тут можна було би використати звичайний line chart

In [ ]:
alt.Chart(df).mark_bar().encode(
    x = alt.X('week day:N', sort='ascending'),
    y = alt.Y('CONSUMPTION:Q', aggregate = 'mean'),
    color = alt.Color('week day:N'),
    tooltip = alt.Tooltip('CONSUMPTION', aggregate = 'mean')
).properties(width = 950, height = 500)